### Phân tích câu và Trực quan hóa

In [1]:
import spacy
from spacy import displacy

# 1. Tải mô hình
nlp = spacy.load("en_core_web_md")

# 2. Phân tích câu
text = "The quick brown fox jumps over the lazy dog."
doc = nlp(text)

# 3. Trả lời câu hỏi trong bài
print(f"ROOT của câu: {[token.text for token in doc if token.dep_ == 'ROOT']}")

print("\nPhân tích từ 'jumps':")
jumps_token = [t for t in doc if t.text == "jumps"][0]
print(f" - Các từ phụ thuộc (Children): {[child.text for child in jumps_token.children]}")


print("\nPhân tích từ 'fox':")
fox_token = [t for t in doc if t.text == "fox"][0]
print(f" - Là con của (Head): {fox_token.head.text}")

# 4. Trực quan hóa (Nếu chạy trên máy local)
# displacy.serve(doc, style="dep")

# Lưu ý: Nếu bạn dùng Jupyter Notebook/Colab, hãy dùng lệnh sau để hiển thị ngay trong dòng:
# displacy.render(doc, style="dep", jupyter=True)

ROOT của câu: ['jumps']

Phân tích từ 'jumps':
 - Các từ phụ thuộc (Children): ['fox', 'over', '.']

Phân tích từ 'fox':
 - Là con của (Head): jumps


ROOT: jumps (Động từ chính).

Dependent của jumps: fox (nsubj - chủ ngữ), over (prep - giới từ), . (punct - dấu câu).

Head của fox: jumps.

### Truy cập các thành phần trong cây phụ thuộc

In [2]:
text_3 = "Apple is looking at buying U.K. startup for $1 billion"
doc_3 = nlp(text_3)

print(f"{'TEXT':<12} | {'DEP':<10} | {'HEAD TEXT':<12} | {'HEAD POS':<8} | {'CHILDREN'}")
print("-" * 75)

for token in doc_3:
    children = [child.text for child in token.children]
    print(f"{token.text:<12} | {token.dep_:<10} | {token.head.text:<12} | {token.head.pos_:<8} | {children}")

TEXT         | DEP        | HEAD TEXT    | HEAD POS | CHILDREN
---------------------------------------------------------------------------
Apple        | nsubj      | looking      | VERB     | []
is           | aux        | looking      | VERB     | []
looking      | ROOT       | looking      | VERB     | ['Apple', 'is', 'at']
at           | prep       | looking      | VERB     | ['buying']
buying       | pcomp      | at           | ADP      | ['startup']
U.K.         | compound   | startup      | NOUN     | []
startup      | dobj       | buying       | VERB     | ['U.K.', 'for']
for          | prep       | startup      | NOUN     | ['billion']
$            | quantmod   | billion      | NUM      | []
1            | compound   | billion      | NUM      | []
billion      | pobj       | for          | ADP      | ['$', '1']


### Duyệt cây phụ thuộc để trích xuất thông tin

#### Bài toán: Tìm chủ ngữ và tân ngữ của một động từ

In [3]:
text_4a = "The cat chased the mouse and the dog watched them."
doc_4a = nlp(text_4a)

print("--- TÌM BỘ BA (S-V-O) ---")
for token in doc_4a:
    if token.pos_ == "VERB":
        verb = token.text
        subject = None
        obj = None

        for child in token.children:
            if child.dep_ == "nsubj":
                subject = child.text
            if child.dep_ in ["dobj", "pobj"]: # Mở rộng thêm pobj nếu cần
                obj = child.text

        if subject and obj:
            print(f"Found Triplet: ({subject}, {verb}, {obj})")

--- TÌM BỘ BA (S-V-O) ---
Found Triplet: (cat, chased, mouse)
Found Triplet: (dog, watched, them)


#### Bài toán: Tìm các tính từ bổ nghĩa cho một danh từ

In [4]:
text_4b = "The big, fluffy white cat is sleeping on the warm mat."
doc_4b = nlp(text_4b)

print("\n--- TÌM TÍNH TỪ BỔ NGHĨA ---")
for token in doc_4b:
    if token.pos_ == "NOUN":
        adjectives = []
        for child in token.children:
            if child.dep_ == "amod":
                adjectives.append(child.text)

        if adjectives:
            print(f"Danh từ '{token.text}' được bổ nghĩa bởi: {adjectives}")


--- TÌM TÍNH TỪ BỔ NGHĨA ---
Danh từ 'cat' được bổ nghĩa bởi: ['big', 'fluffy', 'white']
Danh từ 'mat' được bổ nghĩa bởi: ['warm']


### Bài tập tự luyện

#### Bài 1: Tìm động từ chính của câu

Động từ chính của câu thường có quan hệ ROOT. Viết một hàm
find_main_verb(doc) nhận vào một đối tượng Doc của spaCy và trả về
Token là động từ chính.

In [5]:
def find_main_verb(doc):
    for token in doc:
        # ROOT thường là động từ chính, nhưng đôi khi có thể là danh từ trong câu định danh
        if token.dep_ == "ROOT":
            return token
    return None

# Test
text_ex1 = "She accepted the job offer."
doc_ex1 = nlp(text_ex1)
main_verb = find_main_verb(doc_ex1)
print(f"Động từ chính (ROOT): {main_verb.text} (POS: {main_verb.pos_})")

Động từ chính (ROOT): accepted (POS: VERB)


#### Bài 2: Trích xuất các cụm danh từ (Noun Chunks)
spaCy đã có sẵn thuộc tính .noun_chunks để trích xuất các cụm danh
từ. Tuy nhiên, hãy thử tự viết một hàm để làm điều tương tự.

Gợi ý: Một cụm danh từ đơn giản là một danh từ và tất cả các từ bổ
nghĩa cho nó (như det, amod, compound). Bạn có thể bắt đầu từ một
danh từ và duyệt ngược lên head hoặc duyệt xuống các children của
nó.

In [6]:
def extract_noun_chunks_manual(doc):
    chunks = []
    for token in doc:
        # Bắt đầu từ các danh từ (Noun hoặc Proper Noun)
        if token.pos_ in ["NOUN", "PROPN"]:
            # Lấy các từ con bổ nghĩa (det: mạo từ, amod: tính từ, compound: từ ghép)
            modifiers = [child for child in token.children if child.dep_ in ["det", "amod", "compound"]]

            # Sắp xếp lại theo vị trí xuất hiện trong câu để cụm từ đúng thứ tự
            chunk_tokens = modifiers + [token]
            chunk_tokens.sort(key=lambda x: x.i) # x.i là chỉ số vị trí (index)

            # Ghép thành chuỗi
            chunk_text = " ".join([t.text for t in chunk_tokens])
            chunks.append(chunk_text)
    return chunks

# Test
text_ex2 = "The autonomous electric car crashed into a red wall."
doc_ex2 = nlp(text_ex2)
print(f"Cụm danh từ (Thủ công): {extract_noun_chunks_manual(doc_ex2)}")
# So sánh với spaCy có sẵn
print(f"Cụm danh từ (spaCy): {[chunk.text for chunk in doc_ex2.noun_chunks]}")

Cụm danh từ (Thủ công): ['The autonomous electric car', 'a red wall']
Cụm danh từ (spaCy): ['The autonomous electric car', 'a red wall']


#### Bài 3: Tìm đường đi ngắn nhất trong cây
Viết một hàm get_path_to_root(token) để tìm đường đi từ một token
bất kỳ lên đến gốc (ROOT) của cây. Hàm nên trả về một danh sách
các token trên đường đi.

In [7]:
def get_path_to_root(token):
    path = [token]
    current_token = token

    # Duyệt ngược lên cho đến khi gặp ROOT
    # Điều kiện dừng: token là ROOT (dep_ là ROOT) hoặc head chính là nó
    while current_token.dep_ != "ROOT":
        current_token = current_token.head
        path.append(current_token)

    return path

# Test
text_ex3 = "The quick brown fox jumps over the lazy dog."
doc_ex3 = nlp(text_ex3)
# Thử tìm đường từ 'dog' lên 'jumps'
token_dog = doc_ex3[8] # từ "dog"
path = get_path_to_root(token_dog)

print(f"Đường đi từ '{token_dog.text}' đến ROOT:")
for t in path:
    print(f" -> {t.text} ({t.dep_})")

Đường đi từ 'dog' đến ROOT:
 -> dog (pobj)
 -> over (prep)
 -> jumps (ROOT)
